# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [108]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=newuser password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
song_files

['/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/so

In [8]:
len(song_files)

71

In [9]:
pd.read_json(song_files[5], lines=True).T

,0
num_songs,1
artist_id,ARMJAGH1187FB546F3
artist_latitude,35.14968
artist_longitude,-90.04892
artist_location,"Memphis, TN"
artist_name,The Box Tops
song_id,SOCIWDW12A8C13D406
title,Soul Deep
duration,148.03546
year,1969


In [10]:
filepath = get_files('data/log_data')
filepath


['/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-11-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-23-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-18-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-04-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-01-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-14-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-08-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-26-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/log_data/2018/11/2018-11-25-events.json',
 '/Users/apple/pyprojects/project_data_modeling_postgres/data/lo

In [102]:
df_test = pd.read_json(filepath[0], lines=True)

tuple(df_test[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
# tuple(df_test[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0][1:])

('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

In [13]:
song_files

dfs = []

for file in song_files:
    x = pd.read_json(file, lines=True)
    dfs.append(x)



In [18]:
df_songs = pd.concat(dfs)

df_songs

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0
...,...,...,...,...,...,...,...,...,...,...
0,1,ARAGB2O1187FB3A161,NaN,NaN,,Pucho & His Latin Soul Brothers,SOLEYHO12AB0188A85,Got My Mojo Workin,338.23302,0
0,1,AR62SOJ1187FB47BB5,NaN,NaN,,Chase & Status,SOGVQGJ12AB017F169,Ten Tonne,337.68444,2005
0,1,ARBEBBY1187B9B43DB,NaN,NaN,"Gainesville, FL",Tom Petty,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261,1994
0,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0


In [80]:
# song data

# songs - songs in music database
# song_id, title, artist_id, year, duration

song_data = df_songs[['song_id', 'title', 'artist_id', 'year', 'duration']]

song_data = song_data.drop_duplicates('song_id')

song_data = song_data.set_index('song_id')

song_data = song_data.to_records()

song_data = list(song_data)

song_data

[('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363),
 ('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832),
 ('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608),
 ('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016),
 ('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506),
 ('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546),
 ('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179),
 ('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546),
 ('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, 266.39628),
 ('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, 267.7024),
 ('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, 114.78159),
 ('SOVYKGO12AB0187199', 'Crazy Mixed Up World', 'ARH4Z031187B9A71F2', 1

In [78]:
# artist data

# artist_id, name, location, latitude, longitude

artist_data = df_songs[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

artist_data = artist_data.drop_duplicates('artist_id')

artist_data = artist_data.set_index('artist_id')

artist_data = artist_data.to_records()

artist_data = list(artist_data)

[('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan),
 ('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan),
 ('ARXR32B1187FB57099', 'Gob', '', nan, nan),
 ('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', nan, nan),
 ('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955),
 ('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892),
 ('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan),
 ('ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan),
 ('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan),
 ('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan),
 ('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan),
 ('ARH4Z031187B9A71F2', 'Faye Adams', 'Newark, NJ', 40.73197, -74.17418),
 ('ARB29H41187B98F0EF', 'Terry Callier', 'Chicago', 41.88415, -87.63241),
 ('AR0RCMP1187FB3F427', 'Billie Jo Spears', 'Beaumont, TX', 30.08615, -94.10158),
 ('AR47JEX1187B995D81', 'SUE THOMPSON', 'Nevada, MO', 37.83721, 

In [141]:
pd.read_json(filepath[0], lines=True)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Sidewalk Prophets,Logged In,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,You Love Me Anyway (Album),200,1541970568796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
91,Rise Against,Logged In,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Torches,200,1541970828796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
92,K'Naan,Logged In,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Wavin' Flag,200,1541971049796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
93,Patrick Jumpen,Logged In,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,480,Holiday,200,1541979540796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [160]:
df_log = pd.read_json(filepath[0], lines=True)

df_log = df_log[df_log['page'] == 'NextSong']

df_log['datetime'] = df_log['ts'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))

# start_time, hour, day, week, month, year, weekday

df_time = df_log[['ts', 'datetime']]

df_time['hour'] = df_time['datetime'].apply(lambda x: x.hour)
df_time['day'] = df_time['datetime'].apply(lambda x: x.day)
df_time['week'] = df_time['datetime'].apply(lambda x: x.week)
df_time['month'] = df_time['datetime'].apply(lambda x: x.month)
df_time['year'] = df_time['datetime'].apply(lambda x: x.year)
df_time['weekday'] = df_time['datetime'].apply(lambda x: x.weekday())
#
time_data = df_time[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']].values

time_data

# time_data = list(time_data)
# time_data

# # hour
# df_log.iloc[0].datetime.hour
#
# #
# df_log.iloc[0].datetime.day
#
# #
# df_log.iloc[0].datetime.week
#
# #
# df_log.iloc[0].datetime.month
#
# #
# df_log.iloc[0].datetime.year
#
# # week day
# df_log.iloc[0].datetime.weekday()
#
# tuple()

/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_12062/361532148.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['hour'] = df_time['datetime'].apply(lambda x: x.hour)
/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_12062/361532148.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['day'] = df_time['datetime'].apply(lambda x: x.day)
/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_12062/361532148.py:17: SettingWithCopyWarning: 
A value is trying to be set

array([[1541903636796,             2,            11,            45,
                   11,          2018,             6],
       [1541903770796,             2,            11,            45,
                   11,          2018,             6],
       [1541904034796,             2,            11,            45,
                   11,          2018,             6],
       [1541910973796,             4,            11,            45,
                   11,          2018,             6],
       [1541911006796,             4,            11,            45,
                   11,          2018,             6],
       [1541930188796,             9,            11,            45,
                   11,          2018,             6],
       [1541930447796,            10,            11,            45,
                   11,          2018,             6],
       [1541930647796,            10,            11,            45,
                   11,          2018,             6],
       [1541930908796,  

In [167]:
# load user table
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]

for i, row in user_df.iterrows():
    print(tuple(row))



('69', 'Anabelle', 'Simpson', 'F', 'free')
('69', 'Anabelle', 'Simpson', 'F', 'free')
('69', 'Anabelle', 'Simpson', 'F', 'free')
('32', 'Lily', 'Burns', 'F', 'free')
('75', 'Joseph', 'Gutierrez', 'M', 'free')
('92', 'Ryann', 'Smith', 'F', 'free')
('92', 'Ryann', 'Smith', 'F', 'free')
('92', 'Ryann', 'Smith', 'F', 'free')
('92', 'Ryann', 'Smith', 'F', 'free')
('92', 'Ryann', 'Smith', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('49', 'Chloe', 'Cuevas', 'F', 'free')
('88', 'Mohammad', 'Rodriguez', 'M', 'free')
('88', 'Mohammad', 'Rodriguez', 'M', 'free')
('96', 'Cierra', 'Finley', 'F', 'free')
('26', 'Ryan', 'Smith', 'M', 'free')
('96', 'Cierra', 'Finley', 'F', 'free')
('96', 'Cierra', 'Finley', 'F', 'free')
('44', 'Aleena', 'Kirby', 'F', 'paid')
('9

In [ ]:
# load songplay table
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]

for i, row in user_df.iterrows():
    print(tuple(row))

In [ ]:
df =
df.head()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [30]:
pd.read_json(song_files[5], lines=True)

pd.read_json(song_files[6], lines=True)

pd.concat([pd.read_json(song_files[5], lines=True), pd.read_json(song_files[6], lines=True)])

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [32]:
pd.read_json(song_files[5], lines=True)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


In [31]:
pd.read_json(song_files[6], lines=True)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [33]:
pd.concat([pd.read_json(song_files[5], lines=True), pd.read_json(song_files[6], lines=True)])

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [ ]:
song_data =
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data =
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files =

In [ ]:
filepath =

In [ ]:
df =
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df =
df.head()

In [ ]:
t =
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df =
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df =

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.